In [1]:
# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Training utils
import utils_xgboost

# Optuna
import optuna

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Modelos
from sklearn.linear_model import LinearRegression # for building a linear regression model
from sklearn.svm import SVR # for building SVR model
from sklearn.preprocessing import MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import mean_absolute_error #MAE
from sklearn.metrics import mean_absolute_percentage_error #MAPE
from sklearn.metrics import mean_squared_error #MSE, para RMSE: squared = False

# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

C:\Users\Fer\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# Cargamos los datos

In [2]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates = ['FECHAHORA'])

# Variables de entrenamiento

In [3]:
estacion = 4

variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 
             'TEMPERATURA', 'HUMEDAD', 'PRESION', 'TEMPERATURA_PRONOSTICO', 
             'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO', 'DIA_SEM', 'TRAFICO' , 'AQI_MP10', 'AQI_MP2_5']

dependent = ['AQI_MP2_5']

number_of_features = len(variables)

training_days = 7 
forecast_days = 1 
samples_per_day = 288
step = 288/4

# Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
train_months = relativedelta(months = 12)

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test



# Procesamiento previo a entrenamiento

In [4]:
%%time

X_train, y_train, X_test, y_test = utils_xgboost.get_everything(datos, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)

CPU times: total: 15.3 s
Wall time: 15.3 s


In [5]:
print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)


(1433, 2033)
(1433, 288)
(333, 2033)
(333, 288)


# Prueba antes de usar optuna

In [8]:
%%time

params = {'learning_rate' : 0.025,
          'n_estimators' : 250,
          'max_depth': 2,
          'min_child_weight' : 1,
          'gamma': 0.0,
          'subsample': 0.98,
          'colsample_bytree': 0.98,
          'scale_pos_weight': 0.8,
          'seed': 42,
          'verbosity' : 0}


xgb_model = xgb.XGBRegressor(** params)

trained_xgb_model = MultiOutputRegressor(xgb_model).fit(X_train , y_train)

prediction = trained_xgb_model.predict(X_test)
print('prediction', prediction.shape)
print('test', y_test.shape)

MSE = np.mean((prediction - y_test)**2)
print('RMSE: ', MSE**0.5)
MAE = np.mean(np.abs(prediction-y_test))
print('MAE: ', MAE)

prediction (333, 288)
test (333, 288)
RMSE:  9.781115562652332
MAE:  6.694983114898304
CPU times: total: 1h 56min 35s
Wall time: 10min 16s


In [6]:
%%time

def objective(trial):
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 10, 300),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-5, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
    }
    
    optuna_xgb_model = xgb.XGBRegressor(** params)
    
    trained_xgb_model = MultiOutputRegressor(optuna_xgb_model).fit(X_train , y_train)

    prediction = trained_xgb_model.predict(X_test)

    MSE = np.mean((prediction - y_test)**2)
    RMSE = MSE**0.5
    print('RMSE: ', RMSE)
    MAE = np.mean(np.abs(prediction - y_test))
    print('MAE: ', MAE)
    
    return RMSE

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials = 100)

trial = study.best_trial

[I 2023-01-28 20:10:19,875] A new study created in memory with name: no-name-1787ac92-8806-40f0-93a5-92a43e9d708a
[I 2023-01-28 20:14:50,731] Trial 0 finished with value: 46.48396844731619 and parameters: {'max_depth': 9, 'learning_rate': 0.3337374464151056, 'n_estimators': 130, 'min_child_weight': 3, 'gamma': 0.8882093614663984, 'subsample': 0.015841848096434467, 'colsample_bytree': 0.8370926414146762}. Best is trial 0 with value: 46.48396844731619.


RMSE:  46.48396844731619
MAE:  35.6517350307482


[I 2023-01-28 20:16:18,086] Trial 1 finished with value: 17.943521809806217 and parameters: {'max_depth': 8, 'learning_rate': 0.35432120167676945, 'n_estimators': 65, 'min_child_weight': 8, 'gamma': 0.0015364570022530217, 'subsample': 0.07550576862242127, 'colsample_bytree': 0.010658482625998549}. Best is trial 1 with value: 17.943521809806217.


RMSE:  17.943521809806217
MAE:  13.40249268570035


[I 2023-01-28 20:18:51,417] Trial 2 finished with value: 11.83070446089126 and parameters: {'max_depth': 6, 'learning_rate': 0.03932440008548837, 'n_estimators': 127, 'min_child_weight': 8, 'gamma': 0.0008710945684357212, 'subsample': 0.14272133770629547, 'colsample_bytree': 0.010681248427200423}. Best is trial 2 with value: 11.83070446089126.


...

RMSE:  9.56031196304979
MAE:  6.396480330898438


[I 2023-01-29 05:30:08,255] Trial 98 finished with value: 9.514506587280234 and parameters: {'max_depth': 5, 'learning_rate': 0.03501312736977673, 'n_estimators': 78, 'min_child_weight': 8, 'gamma': 0.00590507825523195, 'subsample': 0.577093202211148, 'colsample_bytree': 0.7936061869990182}. Best is trial 98 with value: 9.514506587280234.


RMSE:  9.514506587280234
MAE:  6.368616052546181


[I 2023-01-29 05:37:17,743] Trial 99 finished with value: 9.546704123037301 and parameters: {'max_depth': 5, 'learning_rate': 0.037449825978534045, 'n_estimators': 80, 'min_child_weight': 8, 'gamma': 0.005015010134387393, 'subsample': 0.8509051385770535, 'colsample_bytree': 0.8983441477768507}. Best is trial 98 with value: 9.514506587280234.


RMSE:  9.546704123037301
MAE:  6.409934523754371
CPU times: total: 4d 6h 45min 41s
Wall time: 9h 26min 57s


In [8]:
import joblib

joblib.dump(study, "optuna_studies/study_XGBOOST_230128_e4_7antes_1prediccion_cada6h.pkl")
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

    max_depth: 5
    learning_rate: 0.03501312736977673
    n_estimators: 78
    min_child_weight: 8
    gamma: 0.00590507825523195
    subsample: 0.577093202211148
    colsample_bytree: 0.7936061869990182
